In [39]:
import numpy as np
import random
import secrets
import io
import scipy

N = 1000000
Alpha = 0.05

## Функція статистичних тестів

In [40]:
def test(seq: np.array, b = 8):
    n = len(seq)

    # Рівномірність
    counts = np.zeros(2**b)
    n_j = n / (2**b)

    for a in seq:
        counts[a] += 1

    stat_U = np.sum((counts - n_j)**2 / n_j)
    quant_U = scipy.stats.chi2.ppf(1 - Alpha, 2**b - 1)
    
    print("Тест на рівноймовірність символів:  " + str(stat_U <= quant_U))

    # Незалежність символів
    pair_counts = np.zeros((2**b, 2**b), dtype=float)

    for i in range(0, n-1, 1):
        pair_counts[seq[i]][seq[i+1]] = pair_counts[seq[i]][seq[i+1]] + 1

    S = 0
    for i in range(0, 2**b):
        for j in range(0, 2**b):
            d = (np.sum(pair_counts[i, :]) * np.sum(pair_counts[:, j]))
            if d != 0:
                S += (pair_counts[i, j] ** 2) / d

    stat_I = n * (S - 1)
    quant_I = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1)**2)

    print("Тест на незалежність символів:  " + str(stat_I <= quant_I))

    # Однорідність послідовності
    r = 200
    interval_counts = np.zeros((r, 2**b), dtype=float)

    for i in range(0, r):
        for j in range(0, n // r):
            c = seq[r*i + j]
            interval_counts[i, c] += 1

    S = 0
    for i in range(0, r):
        for j in range(0, 2**b):
            d = (np.sum(interval_counts[i, :]) * np.sum(interval_counts[:, j]))
            if d != 0:
                S += (interval_counts[i, j] ** 2) / d
    
    stat_H = n * (S - 1)
    quant_H = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1) * (r - 1))

    print("Тест на однорідність послідовності:  " + str(stat_H <= quant_H))


### Генератор Лемера

In [60]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 5  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = np.array(seq % (2**8), dtype=np.uint8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = np.array(seq >> 24, dtype=np.uint8)

        return seq


In [61]:
de1 = Linear_Low()
de2 = Linear_High()

t1 = de1.generate_bytes(N)
t2 = de2.generate_bytes(N)

test(t1)
test(t2)

array([  5, 124, 243, 106, 225,  88, 207,  70, 189,  52, 171,  34, 153,
        16, 135, 254, 117, 236,  99, 218,  81, 200,  63, 182,  45, 164,
        27, 146,   9, 128, 247, 110, 229,  92, 211,  74, 193,  56, 175,
        38, 157,  20, 139,   2, 121, 240, 103, 222,  85, 204,  67, 186,
        49, 168,  31, 150,  13, 132, 251, 114, 233,  96, 215,  78, 197,
        60, 179,  42, 161,  24, 143,   6, 125, 244, 107, 226,  89, 208,
        71, 190,  53, 172,  35, 154,  17, 136, 255, 118, 237, 100, 219,
        82, 201,  64, 183,  46, 165,  28, 147,  10, 129, 248, 111, 230,
        93, 212,  75, 194,  57, 176,  39, 158,  21, 140,   3, 122, 241,
       104, 223,  86, 205,  68, 187,  50, 169,  32, 151,  14, 133, 252,
       115, 234,  97, 216,  79, 198,  61, 180,  43, 162,  25, 144,   7,
       126, 245, 108, 227,  90, 209,  72, 191,  54, 173,  36, 155,  18,
       137,   0, 119, 238, 101, 220,  83, 202,  65, 184,  47, 166,  29,
       148,  11, 130, 249, 112, 231,  94, 213,  76, 195,  58, 17

### Генератор L20

In [ ]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(20)])
print(smp)
de_L20 = L20(smp)

test(de_L20.generate_bits(N), 1)

### Генератор L89

In [ ]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

test(de_L89.generate_bits(N), 1)

### Генератор Вольфрама

In [43]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations) -> np.uint32: 
    return (n >> rotations | n << (32-rotations)) % 2**32

def lcs(n: np.uint32, rotations) -> np.uint32:
    return (n << rotations | n >> (32-rotations)) % 2**32

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))

        return seq
            

In [44]:
de_wolfram = Wolfram(1)

test(de_wolfram.generate_bits(N), 1)

Тест на рівноймовірність символів:  True
Тест на незалежність символів:  True
Тест на однорідність послідовності:  True


### Генератор "Бібліотекар"

In [ ]:
class Librarian:
    def __init__(self, filename):
        file = io.open(filename, mode='r', encoding='utf-8')
        self.text = file.read()

    
    def generate_bytes(self, n: int):
        if len(self.text) < n:
            raise RuntimeError("Nema sliv, odni emotions")

        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = ord(self.text[i])

        return seq
            

In [ ]:
de_Lb = Librarian("fanfiction.txt")

test(de_Lb.generate_bytes(N))

### Генератор BM

In [45]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array(seq < (self.p - 1) / 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array((seq * 256) // (self.p - 1), dtype=np.uint8) 

        return seq
            

In [46]:
p = int("CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3", 16)
a = int("5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356", 16)

de_BM = BM(p, a)

#test(de_BM.generate_bits(N), 1)
test(de_BM.generate_bytes(N))

Тест на рівноймовірність символів:  True
Тест на незалежність символів:  True
Тест на однорідність послідовності:  True


### Генератор Джиффі

In [47]:
class Geffe:
    def __init__(self, x_init: np.array, y_init: np.array, s_init: np.array):
        self.x = x_init # 11 bits, x11 = x0 + x2
        self.y = y_init # 9 bits,  y9 = y0 + y1 + y3 + y4
        self.s = s_init # 10 bits, s10 = s0 + s3

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = (self.s[0] * self.x[0]) ^ ((1 ^ self.s[0]) * self.y[0])
            # Linear Shift
            self.x[0] = self.x[0] ^ self.x[2]
            self.x = np.roll(self.x, -1)
            self.y[0] = self.y[0] ^ self.y[1] ^ self.y[3] ^ self.y[4]
            self.y = np.roll(self.y, -1)
            self.s[0] = self.s[0] ^ self.s[3]
            self.s = np.roll(self.s, -1)

        return seq
            

In [48]:
x = np.array([random.randint(0, 1) for _ in range(11)])
y = np.array([random.randint(0, 1) for _ in range(9)])
s = np.array([random.randint(0, 1) for _ in range(10)])

print(x)
print(y)
print(s)

Ge_generator = Geffe(x, y, s)

test(Ge_generator.generate_bits(N), 1)

[0 0 1 0 0 1 0 0 0 1 1]
[1 0 0 1 0 0 0 0 0]
[0 0 1 1 1 1 0 1 1 0]
Тест на рівноймовірність символів:  True
Тест на незалежність символів:  True
Тест на однорідність послідовності:  True


### Генератор BBS

In [49]:
class BBS:
    def __init__(self, p, q):
        self.n = p*q

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % (2**8), dtype=np.uint8) 

        return seq
            

In [50]:
p = int("D5BBB96D30086EC484EBA3D7F9CAEB07", 16)
q = int("425D2B9BFDB25B9CF6C416CC6E37B59C1F", 16)

de_BBS = BBS(p, q)

test(de_BBS.generate_bytes(N))
#test(de_BBS.generate_bits(N), 1)

Тест на рівноймовірність символів:  True
Тест на незалежність символів:  False
Тест на однорідність послідовності:  True


### Вбудовані генератори

#### Криптографічно нестійкий генератор

In [56]:
#unsafe_bits_seq = np.array([random.randint(0, 1) for _ in range(N)], dtype=np.uint8)
#test(unsafe_bits_seq, 1)

unsafe_bytes_seq = np.array(list(random.randbytes(N)), dtype=np.uint8)
test(unsafe_bytes_seq)

Тест на рівноймовірність символів:  True
Тест на незалежність символів:  True
Тест на однорідність послідовності:  True


#### Криптографічно стійкий генератор

In [55]:
#safe_bits_seq = np.array([secrets.randbelow(2) for _ in range(N)], dtype=np.uint8)
#test(safe_bits_seq, 1)

safe_bytes_seq = np.array(list(secrets.token_bytes(N)), dtype=np.uint8)
test(safe_bytes_seq)

Тест на рівноймовірність символів:  True
Тест на незалежність символів:  True
Тест на однорідність послідовності:  True
